<a href="https://colab.research.google.com/github/Vishnu-916/PRODIGY_GA_02/blob/main/Neural%20Style%20Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.models import vgg19, VGG19_Weights

import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Desired size of the output image
imsize = 512 if torch.cuda.is_available() else 128  # use small size if no GPU

loader = transforms.Compose([
    transforms.Resize(imsize),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor


def image_loader(image_name):
    image = Image.open(image_name).convert('RGB')
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)


# In Colab, upload your images first:
from google.colab import files
uploaded = files.upload()

# Assuming you upload two files, the first is content, the second is style
# You can adjust this based on the order you upload or if you have specific names
if len(uploaded) >= 2:
  file_names = list(uploaded.keys())
  content_img_name = file_names[0]
  style_img_name = file_names[1]
elif len(uploaded) == 1:
  # If only one file uploaded, assume it's content and use a placeholder for style
  # You might need to modify this logic based on your use case
  content_img_name = list(uploaded.keys())[0]
  style_img_name = None # Or set to a default style image if available
  print("Only one image uploaded. Please upload both content and style images.")
else:
  content_img_name = None
  style_img_name = None
  print("No images uploaded. Please upload content and style images.")


if content_img_name and style_img_name:
  style_img = image_loader(style_img_name)
  content_img = image_loader(content_img_name)

  assert style_img.size() == content_img.size(), \
      "We need to import style and content images of the same size"

unloader = transforms.ToPILImage()  # reconvert into PIL image

plt.ion()

def imshow(tensor, title=None):
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)      # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated

# Display original images
if content_img_name and style_img_name:
    plt.figure()
    imshow(style_img, title='Style Image')
    plt.figure()
    imshow(content_img, title='Content Image')

class ContentLoss(nn.Module):

    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input


def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a feature map (N=c*d)

    features = input.view(a * b, c * d)  # resize F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each column
    return G.div(a * b * c * d)


class StyleLoss(nn.Module):

    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input


cnn = vgg19(weights=VGG19_Weights.DEFAULT).features.to(device).eval()

# normalization for VGG
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)


class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly multiply with input
        # mean: feature mean of one channel of the input
        # std:  feature standard deviation of one channel of the input
        self.mean = mean.view(-1, 1, 1)
        self.std = std.view(-1, 1, 1)

    def forward(self, img):
        # normalize img: subtract mean and divide standard deviation
        return (img - self.mean) / self.std


# create a module to normalize input image so we can easily put it in a
# nn.Sequential
normalization = Normalization(cnn_normalization_mean, cnn_normalization_std).to(device)

# desired depth layers to compute style/content losses :
# content_layers_default = ['conv_4_2']
# style_layers_default = ['conv_1_1', 'conv_2_1', 'conv_3_1', 'conv_4_1', 'conv_5_1']
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']


def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # the contents / style of the input target will be stored in these
    content_losses = []
    style_losses = []

    # these are the layers used for the content and style losses
    # we'll get the target content and style representations of the input
    # image at these layers
    model = nn.Sequential(normalization).to(device)

    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place code has been modified
            # so that it can be run on both GPU and CPU
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(
                layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}_{}".format(i, name), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}_{}".format(i, name), style_loss)
            style_losses.append(style_loss)

    # now we trim off the layers after the last content and style losses
    # (since the model has already calculated the losses for these layers),
    # which would otherwise affect the forward method of the model
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses


# Get the style and content losses
if content_img_name and style_img_name:
    model, style_losses, content_losses = get_style_model_and_losses(
        cnn, cnn_normalization_mean, cnn_normalization_std,
        style_img, content_img)

    # Next, we initialize the input image. To do this, we use the content image
    # as the base image and start optimizing from there.
    input_img = content_img.clone().requires_grad_(True)
    model(input_img)
    print("Done loading model!")

    # We also add an optimizer to the image, which will update the image during
    # the optimization process.
    optimizer = optim.LBFGS([input_img])

    # Hyperparameters
    num_steps = 300  # Number of optimization steps
    style_weight = 1e6  # Weight for style loss
    content_weight = 1   # Weight for content loss

    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values
            with torch.no_grad():
                input_img.clamp_(0, 1)

            optimizer.zero_grad()

            # Forward pass
            model(input_img)

            # Compute the style and content losses
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            # Weighted sum of losses
            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                print('[Run {}] Style Loss: {:4f} Content Loss: {:4f}'.format(
                    run[0], style_score.item(), content_score.item()))

            return loss

        optimizer.step(closure)

        # Display intermediate results
        with torch.no_grad():
            # correct the values
            input_img.clamp_(0, 1)

            plt.figure()
            imshow(input_img)

    # Final display
    plt.figure()
    imshow(style_img, title='Style Image')
    plt.figure()
    imshow(content_img, title='Content Image')

    # remove the fake batch dimension
    input_img = input_img.squeeze(0)
    with torch.no_grad():
        # correct the values
        input_img.clamp_(0, 1)

    plt.figure()
    imshow(input_img, title='Output Image')

    # Save the output
    output_img = unloader(input_img.cpu())
    output_img.save('output.jpg')
    print("Output saved as 'output.jpg'")
    plt.ioff()
    plt.show()
else:
    print("Cannot proceed without both content and style images.")
    plt.ioff()
    plt.show()

Using device: cuda


Saving 8642182.jpg to 8642182.jpg
Only one image uploaded. Please upload both content and style images.
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


100%|██████████| 548M/548M [00:03<00:00, 147MB/s]


Cannot proceed without both content and style images.
